# Intro to M-estimators

#### LICENSE
These notes are released under the 
"Creative Commons Attribution-ShareAlike 4.0 International" license. 
See the **human-readable version** [here](https://creativecommons.org/licenses/by-sa/4.0/)
and the **real thing** [here](https://creativecommons.org/licenses/by-sa/4.0/legalcode). 

#### INSTALLATION instructions

To use this noteboook you may need to install a few packages in `R`:
```
install.packages(c('rmutil', 'robustbase', 'RobStatTM'))
```

## Intro

In this notebook we will review simple location M-estimators, some of their 
robustness properties, and algorithms to compute them. 

We first start by loading a simple data set `robustbase::cushny`. Refer to 
`help(cushny, package='robustbase')` for information on the data. 

In [ ]:
x <- robustbase::cushny

It is always a good idea to look at the data

In [ ]:
boxplot(x, col='tomato3', cex=1.5, pch=19)

In [ ]:
rbind(mean = mean(x), median = median(x))

We now compute an M-estimator, using a Huber loss, and without standardizing. We 
write our own code, using the iterative weighted averages algorithm discussed in class. 

First we define the derivative of the loss function rho (which, in the robustness literature, is often called "Psi"):

In [ ]:
HuberPsi <- function(r, cc)
    return( pmin(pmax(-cc, r), cc) )

The function `mest0` below takes as arguments the data, the tuning constant `c` (the place where the loss functions transitions from a squared loss to a linear one), a initial estimator "mu(0)" to start the iterations, the maximum number of allowed iterations (default of `100`) and the default convergence tolerance (the algorithm will stop when two consecutive values of the estimated mu differ in less than `eps`): 

In [ ]:
mest0 <- function(x, cc=1.345, init=median(x), max.it = 100, eps=1e-8) {
    m1 <- init
    m0 <- m1 + 10*eps
    it <- 0
    while( ((it <- it+1) < max.it ) & (abs(m1-m0) > eps ) ) {
        re <- (x - m1)
        w <- HuberPsi(re, cc=cc)/re
        w[ is.na(w) ] <- 1
        m0 <- m1
        m1 <- sum( x*w ) / sum(w)
    }
    return(m1)
}

We use the function on the data in `x` to compute the M-estimator (note that we let all the other arguments take their default values):

In [ ]:
(mu0 <- mest0(x))

Note that the M-estimator is actually "between" the mean and the median. 

It is always a good habit to perform a quick sanity check and verify that the estimator satisfies the estimating equations (in this case, that the first order conditions are met): 

In [ ]:
mean( HuberPsi(x-mu0, cc=1.345)) # this should be essentially zero

## Lack of scale invariance, robustness

As we discussed in class, this estimator is not scale equivariant. For example, if we divide all the data by 100, and then multiply the resulting estimator by 100, we do not recover the original estimator. In fact, something much more "surprising" happens:

In [ ]:
rbind(mean=c(mean(x), mean(x/100)*100),
      median=c(median(x), median(x/100)*100),
      Mest=c(mest0(x), mest0(x/100)*100))

The suppossedly robust M-estimator computed on the "scaled" data is identical to the sample mean! This is a serious problem, as the estimator is not robust any longer. As discussed in class, the problem is that the tuning parameter (the choice of loss function rho depends on the "size" of the data / residuals). 

We now add 2 outliers to illustrate that this non-scale-equivariant M-estimator really is not robust.

In [ ]:
xc <- c(x, rnorm(2, mean=5.5, sd=.5))

We now compute the estimators again. Note that the performance of the M-estimator deteriorates (it appears to be affected by the outliers), but not as much as the sample mean.  

In [ ]:
rbind(mean=c(mean(x), mean(xc)),
      median=c(median(x), median(xc)),
      Mest=c(mest0(x), mest0(xc)))

To again illustrate the problem of the relative magnitudes of the data and the tuning constant of the (hopefully) robust loss, we compute the estimators on "proportionally smaller" data, and then re-scale it back to the original units:

In [ ]:
rbind(mean=c(mean(x), mean(xc), mean(xc/100)*100),
      median=c(median(x), median(xc), median(xc/100)*100),
      Mest=c(mest0(x), mest0(xc), mest0(xc/100)*100))

Now we can clearly see the deterioration of the M-estimator. It is just not working well. 

### Another illustration of this problem (synthetic)

We can also see the problem directly on data that have a much smaller scale than the default tuning parameter we use above. The following is a simple synthetic example. Generate a sample of 25 observations from a N(0.1, 0.01) distribution (variance = 0.01), and add 3 outliers located around 1 (note that they are located 0.9 / 0.1 = 9 standard deviations away from the true mean!) 

In [ ]:
set.seed(123)
x2 <- rnorm(25, mean=0.1, sd=.1)
x2c <- c(x2, rnorm(5, mean=1, sd=.1))

Now, compare the sample mean, median and the "robust" non-scale-equivariant M-estimator, on the clean and contaminated data sets:

In [ ]:
rbind(mean=c(mean(x2), mean(x2c)),
      median=c(median(x2), median(x2c)),
      Mest=c(mest0(x2), mest0(x2c)))

Note how for this data the M-estimator is **identical** to the **sample mean**! 

Can you explain why this is, in fact, to be expected here? 

## Using scaled residuals helps in choosing the robust loss

The solution, as we discussed in more detail in class, is to use standardized residuals. The only difference between the "good" M-estimator computed with `mest` below and the previous one (`mest0`) is the inclusion of the robust scale estimator (`si <- mad(x)`), and its use in the computation of residuals (`re <- (x - m1) / si`):

In [ ]:
mest <- function(x, cc=1.345, init=median(x), si = mad(x), max.it = 100, eps=1e-8) {
    m1 <- init
    m0 <- m1 + 10*eps
    it <- 0
    while( ((it <- it+1) < max.it ) & (abs(m1-m0) > eps ) ) {
        re <- (x - m1) / si
        w <- HuberPsi(re, cc=cc)/re
        w[ is.na(w) ] <- 1
        m0 <- m1
        m1 <- sum( x*w ) / sum(w)
    }
    return(m1)
}

And now everything works fine!

In [ ]:
rbind(mean=c(mean(x), mean(xc), mean(xc/100)*100),
      median=c(median(x), median(xc), median(xc/100)*100),
      Mest=c(mest(x), mest(xc), mest(xc/100)*100))

Sanity check again. First order conditions:

In [ ]:
si <- mad(xc)
mu1 <- mest(xc)
mean( HuberPsi((xc-mu1)/si, cc=1.345))

It also works fine for the artificial example:

In [ ]:
rbind(mean=c(mean(x2), mean(x2c)),
      median=c(median(x2), median(x2c)),
      Mest=c(mest(x2), mest(x2c)))

## M-estimators are robust, not immutable

Note, however, that the M-estimator is in fact, affected by the outliers. Fortunately, this effect is bounded, and will not get any worse even if the outliers were much more extreme. For example, if the outliers were placed at `+20` (instead of `5.5`)

In [ ]:
xc2 <- c(x, rnorm(2, mean=20, sd=.5))

... then the M-estimator does not shift any further to the right, as opposed to what happens with the sample mean: 

In [ ]:
rbind(mean=c(mean(x), mean(xc), mean(xc2)),
      median=c(median(x), median(xc), median(xc2)),
      Mest=c(mest(x), mest(xc), mest(xc2)))

## M-estimators with a bounded loss function

M-estimators with a bounded loss (re-descending score function) have a different behaviour, and usually offer better bias performance (they deviate less than Huber-type M-estimators) when outliers grow. 

Bounded (but not constant, and thus necessarily non-convex) loss functions will play a key role in the linear regression context. Note that a bounded loss function has a derivative (score function) that becomes zero for large residuals (instead of constant for Huber-type loss functions). 

A commonly used family of such loss / score functions is Tukey's bisquare family:

In [ ]:
TukeyPsi <- function(r, cc) {
    tmp <- r/cc
    tmp2 <- tmp*(1-tmp^2)^2
    tmp2[ abs(tmp) > 1 ] <- 0
    return(tmp2)
}
xx <- seq(-3, 3, length=1000)
plot(xx, TukeyPsi(xx, cc=2.5), type='l', lwd=4, col='hotpink',
    ylab = expression(Psi[c]), cex.lab=1.5, xlab='t')
abline(h=0); abline(v=0)

The corresponding weight function (associated with the iterative re-weighted averages algorithm) is also qualitatively different from Huber's:

In [ ]:
TukeyW <- function(r, cc) {
    tmp <- TukeyPsi(r, cc) / r
    tmp[is.na(tmp)] <- (1/cc)
    return(tmp)
}
xx <- seq(-3, 3, length=1000)
plot(xx, TukeyW(xx, cc=2.5), type='l', lwd=4, col='seagreen4',
    ylab = expression(Psi[c]), cex.lab=1.5, xlab='t')
abline(h=0); abline(v=0)

What tuning constant should we use to obtain 95% efficiency if the data are Gaussian? Compute the asymptotic variance and pick `cc` so that when the data are Gaussian it is (approximately) 1.05:

In [ ]:
TukeyPsiprime <- function(r, cc) {
    tmp <- r/cc
    tmp2 <- (1/cc)*(1-tmp^2)*(1-5*tmp^2)
    tmp2[ abs(tmp) > 1 ] <- 0
    return(tmp2)
}

In [ ]:
f1 <- function(r, cc) dnorm(r)*TukeyPsi(r, cc)^2
f2 <- function(r, cc) dnorm(r)*TukeyPsiprime(r, cc)
a1 <- integrate(f1, lower=-Inf, upper=+Inf, cc=4.65)
a2 <- integrate(f2, lower=-Inf, upper=+Inf, cc=4.65)
a1$message
a2$message
a1$value / (a2$value^2)

Now adapt the `mest` function to use Tukey's bisquare loss function, call it `mest2`:

In [ ]:
mest2 <- function(x, cc=4.65, init=median(x), si = mad(x), max.it = 100, eps=1e-8) {
    m1 <- init
    m0 <- m1 + 10*eps
    it <- 0
    while( ((it <- it+1) < max.it ) & (abs(m1-m0) > eps ) ) {
        re <- (x - m1) / si
        w <- TukeyPsi(re, cc=cc)/re
        w[ is.na(w) ] <- 1
        m0 <- m1
        m1 <- sum( x*w ) / sum(w)
    }
    return(m1)
}

A simple example of the "spring" behaviour. We build a "clean" data set from a N(2, 1/4) distribution, and add outliers centred around 4.5 (this is the object `xc` below), and also another sample where outliers were added around 9 (`xc2`):

In [ ]:
set.seed(123)
x <- rnorm(40, mean=2, sd=.5)
xc <- c(x, rnorm(15, mean=5, sd=.25))
xc2 <- c(x, rnorm(15, mean=10, sd=.25))
boxplot(x, xc, xc2, col=c('steelblue', 'seagreen2', 'tomato3'),
       names=c('Clean', 'Outs @ 5', 'Outs @ 10'))

In [ ]:
a <- rbind(mean=c(mean(x), mean(xc), mean(xc2)),
      median=c(median(x), median(xc), median(xc2)),
      Mest=c(mest(x), mest(xc), mest(xc2)),
      RedMest = c(mest2(x), mest2(xc), mest2(xc2)))
colnames(a) <- c('Clean', 'Outs @ 5', 'Outs @ 10')
a